In [ ]:
pip install bs4

In [ ]:
pip install tqdm

In [ ]:
pip install requests

In [ ]:
pip install BeautifulSoup4

In [ ]:
pip install lxml

In [16]:
from bs4 import BeautifulSoup
import requests
import time
import datetime
import smtplib
from tqdm import tqdm
import pandas as pd
import webbrowser 
import time

In [19]:


res = requests.get("http://en.wikipedia.org/wiki/List_of_S%26P_500_companies")                                                  
soup = BeautifulSoup(res.text, 'html.parser') #if you find any problem with "lxml" then try using "html.parser" instead
table = soup.find("table",class_="wikitable sortable")
for items in table.find_all("tr")[:-1]:
    data = [' '.join(item.text.split()) for item in items.find_all(['th','td'])]
    print(data)                                                             
                                                             

['Symbol', 'Security', 'SEC filings', 'GICS Sector', 'GICS Sub-Industry', 'Headquarters Location', 'Date added', 'CIK', 'Founded']
['MMM', '3M', 'reports', 'Industrials', 'Industrial Conglomerates', 'Saint Paul, Minnesota', '1976-08-09', '0000066740', '1902']
['AOS', 'A. O. Smith', 'reports', 'Industrials', 'Building Products', 'Milwaukee, Wisconsin', '2017-07-26', '0000091142', '1916']
['ABT', 'Abbott', 'reports', 'Health Care', 'Health Care Equipment', 'North Chicago, Illinois', '1964-03-31', '0000001800', '1888']
['ABBV', 'AbbVie', 'reports', 'Health Care', 'Pharmaceuticals', 'North Chicago, Illinois', '2012-12-31', '0001551152', '2013 (1888)']
['ACN', 'Accenture', 'reports', 'Information Technology', 'IT Consulting & Other Services', 'Dublin, Ireland', '2011-07-06', '0001467373', '1989']
['ATVI', 'Activision Blizzard', 'reports', 'Communication Services', 'Interactive Home Entertainment', 'Santa Monica, California', '2015-08-31', '0000718877', '2008']
['ADM', 'ADM', 'reports', 'Con

In [24]:
tickers = []
companies = []
industries = []
headquarters = []
dates_1st_added = []
years_founded = []

for row in tqdm(table.findAll('tr')[1:]):
    ticker = row.findAll('td')[0].text
    company = row.findAll('td')[1].text
    industry = row.findAll('td')[3].text
    headquarter = row.findAll('td')[5].text
    date_1st_added = row.findAll('td')[6].text
    year_founded = row.findAll('td')[8].text

    tickers.append(ticker)
    companies.append(company)
    industries.append(industry)
    headquarters.append(headquarter)
    dates_1st_added.append(date_1st_added)
    years_founded.append(year_founded)


100%|██████████| 503/503 [00:00<00:00, 5109.16it/s]


In [25]:


df = pd.DataFrame(list(zip(tickers, companies, industries, headquarters, dates_1st_added, years_founded)), 
                  columns =['Symbol', 'Company', 'Sector', 'Headquarter', 'Year First Added', 'Foundation'])
df = df.reset_index(drop=True)
df



,Symbol,Company,Sector,Headquarter,Year First Added,Foundation
0,MMM\n,3M,Industrials,"Saint Paul, Minnesota",1976-08-09,1902\n
1,AOS\n,A. O. Smith,Industrials,"Milwaukee, Wisconsin",2017-07-26,1916\n
2,ABT\n,Abbott,Health Care,"North Chicago, Illinois",1964-03-31,1888\n
3,ABBV\n,AbbVie,Health Care,"North Chicago, Illinois",2012-12-31,2013 (1888)\n
4,ACN\n,Accenture,Information Technology,"Dublin, Ireland",2011-07-06,1989\n
...,...,...,...,...,...,...
498,YUM\n,Yum! Brands,Consumer Discretionary,"Louisville, Kentucky",1997-10-06,1997\n
499,ZBRA\n,Zebra Technologies,Information Technology,"Lincolnshire, Illinois",2019-12-23,1969\n
500,ZBH\n,Zimmer Biomet,Health Care,"Warsaw, Indiana",2001-08-07,1927\n
501,ZION\n,Zions Bancorporation,Financials,"Salt Lake City, Utah",2001-06-22,1873\n


In [26]:
def clean_columns(x):
    return x.replace("\n", "")

def clean_foundation(x):
    return x.split('(')[-1].split(')')[0][-4:]

def get_state_country(x):
    return x.split(', ')[-1]


In [27]:


df["Symbol"] = df["Symbol"].apply(lambda x: clean_columns(x))                     # Clean "\n"
df["Headquarter"] = df["Headquarter"].apply(lambda x: get_state_country(x))       # Get US state or country
df["Year First Added"] = df["Year First Added"].apply(lambda x: clean_columns(x)) # Clean "\n"
df["Year First Added"] = df["Year First Added"].apply(lambda x: x[:4])            # Keep the year of date first added
df["Year First Added"] = df["Year First Added"].replace([''], 0).astype(int)      # Convert column to integer after change empty values to 0
df["Foundation"] = df["Foundation"].apply(lambda x: clean_columns(x))             # Clean "\n"
df["Foundation"] = df["Foundation"].apply(lambda x: int(clean_foundation(x)))     # Keep foundation year in order to convert to int
df



,Symbol,Company,Sector,Headquarter,Year First Added,Foundation
0,MMM,3M,Industrials,Minnesota,1976,1902
1,AOS,A. O. Smith,Industrials,Wisconsin,2017,1916
2,ABT,Abbott,Health Care,Illinois,1964,1888
3,ABBV,AbbVie,Health Care,Illinois,2012,1888
4,ACN,Accenture,Information Technology,Ireland,2011,1989
...,...,...,...,...,...,...
498,YUM,Yum! Brands,Consumer Discretionary,Kentucky,1997,1997
499,ZBRA,Zebra Technologies,Information Technology,Illinois,2019,1969
500,ZBH,Zimmer Biomet,Health Care,Indiana,2001,1927
501,ZION,Zions Bancorporation,Financials,Utah,2001,1873


In [30]:
df.to_csv('s&p500.csv', index=False)